<a href="https://colab.research.google.com/github/obaidah3/robot-hallway-mdp/blob/main/MDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [2]:
# States and actions
states = [0, 1, 2, 3]
actions = ["LEFT", "RIGHT"]

In [3]:
# Discount factor (افتراض)
gamma = 0.9

In [4]:
# Rewards
def reward(state, action, next_state):
    if next_state == 3:
        return 10
    else:
        return -1

Transition probabilities

In [5]:
def transition(state, action):
    if state == 3:
        return [(3, 1.0)]  # terminal

    if action == "RIGHT":
        intended = min(state + 1, 3)
    else:  # LEFT
        intended = max(state - 1, 0)

    return [
        (intended, 0.8),
        (state, 0.2)
    ]

Value Iteration

In [6]:
# Initialize values
V = {s: 0 for s in states}

theta = 1e-4  # convergence threshold

while True:
    delta = 0
    for s in states:
        if s == 3:
            continue

        values = []
        for a in actions:
            total = 0
            for next_s, prob in transition(s, a):
                r = reward(s, a, next_s)
                total += prob * (r + gamma * V[next_s])
            values.append(total)

        best_value = max(values)
        delta = max(delta, abs(V[s] - best_value))
        V[s] = best_value

    if delta < theta:
        break


In [7]:
policy = {}

for s in states:
    if s == 3:
        policy[s] = "TERMINAL"
        continue

    best_action = None
    best_value = -1e9

    for a in actions:
        total = 0
        for next_s, prob in transition(s, a):
            r = reward(s, a, next_s)
            total += prob * (r + gamma * V[next_s])

        if total > best_value:
            best_value = total
            best_action = a

    policy[s] = best_action

print("Optimal Values:")
for s in states:
    print(f"State {s}: {V[s]:.2f}")

print("\nOptimal Policy:")
for s in states:
    print(f"State {s}: {policy[s]}")


Optimal Values:
State 0: 5.04
State 1: 7.13
State 2: 9.51
State 3: 0.00

Optimal Policy:
State 0: RIGHT
State 1: RIGHT
State 2: RIGHT
State 3: TERMINAL


In [8]:
import random

def simulate(start_state, max_steps=20):
    s = start_state
    steps = 0
    print(f"Start at {s}")

    while s != 3 and steps < max_steps:
        a = policy[s]
        next_states = transition(s, a)
        s = random.choices(
            [ns for ns, _ in next_states],
            [p for _, p in next_states]
        )[0]
        print(f"Action: {a} → New state: {s}")
        steps += 1

    print("Reached charging station!\n")

# Test from all start positions
for start in [0, 1, 2]:
    simulate(start)


Start at 0
Action: RIGHT → New state: 1
Action: RIGHT → New state: 2
Action: RIGHT → New state: 3
Reached charging station!

Start at 1
Action: RIGHT → New state: 1
Action: RIGHT → New state: 2
Action: RIGHT → New state: 3
Reached charging station!

Start at 2
Action: RIGHT → New state: 3
Reached charging station!

